## Unsloth SFT

#### Imports and configs

In [1]:
from unsloth import FastLanguageModel
import torch
import os
from huggingface_hub import login
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset
from unsloth.chat_templates import standardize_sharegpt
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/pressprexx/miniconda3/envs/unsloth/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
import wandb

wandb.login()

%env WANDB_WATCH=all
%env WANDB_SILENT=true

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
mytoken = os.environ["HUGGINGFACE_TOKEN"]
login(token=mytoken)

In [4]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.
load_in_8bit = True

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-0.5B-Instruct", # "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    load_in_8bit = load_in_8bit,
    full_finetuning = True
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: You selected full finetuning support, but 4bit / 8bit is enabled - disabling LoRA / QLoRA.
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA GeForce RTX 4060 Ti. Num GPUs = 1. Max memory: 15.697 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using bfloat16 full finetuning which cuts memory usage by 50%.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [5]:
# model = FastLanguageModel.get_peft_model(
#     model,
#     r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
#     target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
#                       "gate_proj", "up_proj", "down_proj",],
#     lora_alpha = 64,
#     lora_dropout = 0, # Supports any, but = 0 is optimized
#     bias = "none",    # Supports any, but = "none" is optimized
#     # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
#     use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
#     random_state = 3407,
#     use_rslora = False,  # We support rank stabilized LoRA
#     loftq_config = None, # And LoftQ
# )

#### Data Prep


In [ ]:
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

dataset = load_dataset("EdwardSJ151/mario-1-2", split = "train")

In [7]:
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [8]:
dataset[5]["conversations"]

[{'content': 'Give me a level that includes some pipes, no special enemies, a few ground blocks, a few hard blocks, some coin blocks, a few breakable blocks, no koopas, many goombas, a few powerups, no coins, high elevation, Medium difficulty, overworld level',
  'role': 'user'},
 {'content': '--------------------------------------------------\n--------------------------------------------------\n--------------------------------------------------\n--------------------------------------------------\n------------------------SSSSSS--------------------\n--------------------------------------------------\n--------------------------------------------------\n--------------------------------------------------\n---------------?------S!S----S!S------------------\n----------##----------------------<>-------<>-----\n----------##------<>--------------[]-------[]-----\n----------##------[]--g-g---------[]-g-g-g-[]-----\nXXXXXXXXXXXX--XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX-----\nXXXXXXXXXXXX--XXXXXXXXXXXXXXX

In [9]:
dataset[5]["text"]

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nGive me a level that includes some pipes, no special enemies, a few ground blocks, a few hard blocks, some coin blocks, a few breakable blocks, no koopas, many goombas, a few powerups, no coins, high elevation, Medium difficulty, overworld level<|im_end|>\n<|im_start|>assistant\n--------------------------------------------------\n--------------------------------------------------\n--------------------------------------------------\n--------------------------------------------------\n------------------------SSSSSS--------------------\n--------------------------------------------------\n--------------------------------------------------\n--------------------------------------------------\n---------------?------S!S----S!S------------------\n----------##----------------------<>-------<>-----\n----------##------<>--------------[]-------[]-----\n----------##------[]--g-g----

#### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
project_name = "mario_llm_finetunning" 
wandb.init(entity="edwardsj_151", project=project_name, name="qwen-2.5-14b-horizontal-newline-1epoch-teste5")

In [10]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.05,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = None,
        # max_steps = 200,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [11]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_end|>\n<|im_start|>assistant\n",
)

In [12]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4060 Ti. Max memory = 15.697 GB.
1.203 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()
wandb.finish()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 17,071 | Num Epochs = 2 | Total steps = 4,268
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 494,032,768/494,032,768 (100.00% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,2.929500
2,3.501900
3,2.771300
4,2.859200
5,2.627700
6,2.773600
7,2.663800
8,3.071300
9,2.795500
10,2.592300


In [14]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1995.3019 seconds used for training.
33.26 minutes used for training.
Peak reserved memory = 3.771 GB.
Peak reserved memory for training = 2.568 GB.
Peak reserved memory % of max memory = 24.024 %.
Peak reserved memory for training % of max memory = 16.36 %.


#### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Unsloth_Studio.ipynb)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [15]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "I want a level with many pipes, some coin blocks, some goombas, no koopas"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 512, use_cache = True,
                         temperature = 1.2, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nI want a level with many pipes, some coin blocks, some goombas, no koopas<|im_end|>\n<|im_start|>assistant\n--------------------------------------------------\n--------------------------------------------------\n--------------------------------------------------\n-----------------------g--------------------------\n--------------()------SSSSSSSS---SSS!--------------#\n--------------[]---------------------------------\n--------------[]---------------------------------\n--------##----##--------------------------XXXXXX-S!\n-------###----##-------------------------XXXXXXX-S!\n------####--##--------------------------XXXXXXXXX-\n-----#####--##--------------------------XXXXXXXXX-\n----######--##---------------------XXXXXXXXXXXX-X-\n---#######--##---------------------XXXXXXXXXXXX-X-\n--########----------------------------XXXXXXXXXX\n-XXXXXXXXXXXXXX---------------------XXXXXXXX

In [ ]:
print("--------------------------------------------------\n--------------------------------------------------\n--------------------------------------------------\n-----------------------g--------------------------\n--------------()------SSSSSSSS---SSS!--------------#\n--------------[]---------------------------------\n--------------[]---------------------------------\n--------##----##--------------------------XXXXXX-S!\n-------###----##-------------------------XXXXXXX-S!\n------####--##--------------------------XXXXXXXXX-\n-----#####--##--------------------------XXXXXXXXX-\n----######--##---------------------XXXXXXXXXXXX-X-\n---#######--##---------------------XXXXXXXXXXXX-X-\n--########----------------------------XXXXXXXXXX\n-XXXXXXXXXXXXXX---------------------XXXXXXXXXXXX-X-\n-XXXXXXXXXXXXXX---------------------XXXXXXXXXXXX-X-(\n-XXXXXXXXXXXXXX----#--------------------XXXXXXXXX[\n-XXXXXXXXXXXXXXXXX---------------------------XXX[\n-XXXXXXXXXXXXXXXXX---------------------------XXX[[\n-XXXXXXXXXXXXXXXXX--------------------------XXXXX[\n-XXXXXXXXXXXXXXXXX--------------------------XXXXX[\n-XXXXXXXXXXXXXXXXX--------------------------XXXXX[\n-XXXXXXXXXXXXXXXXX-------------------------XXXXX[\n-XXXXXXXXXXXXXXXXX------------------------XXXXXX[\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX---------------------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX----XXX-XXXX---------XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX----XXX-XXXX---------k-XXXXXXXXXX\n-XXXXXXXXXXXXXXXXX----XXX-XXXXXX-XXXXXXXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX----XXX-XXXXXX-XXXXXXXXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX----XXX-XXXXXX-XXXXXXXXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX----XXX-XXXXXX-XXXXXXXXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX----XXX-XXXXXX-XXXXXXXXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX----XXX-XXXXXX-XXXXXXXXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX----XXX-XXXXXX-XXXXXXXXXXXXXXXXX\n-XXXXXXXXXXXXXXXXX----XXX-XXXXXX-XXXXXXXXXXXXXXXXX")

#### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit, scroll down!

In [16]:
save_model_name = "Qwen2.5-0.5B-Instruct-bnb-4bit-mariogpt-teste2"

model.save_pretrained(save_model_name)  # Local saving
tokenizer.save_pretrained(save_model_name)
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('Qwen2.5-0.5B-Instruct-bnb-4bit-mariogpt-teste2/tokenizer_config.json',
 'Qwen2.5-0.5B-Instruct-bnb-4bit-mariogpt-teste2/special_tokens_map.json',
 'Qwen2.5-0.5B-Instruct-bnb-4bit-mariogpt-teste2/vocab.json',
 'Qwen2.5-0.5B-Instruct-bnb-4bit-mariogpt-teste2/merges.txt',
 'Qwen2.5-0.5B-Instruct-bnb-4bit-mariogpt-teste2/added_tokens.json',
 'Qwen2.5-0.5B-Instruct-bnb-4bit-mariogpt-teste2/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [17]:
raise NotImplementedError("Para aqui")

NotImplementedError: Para aqui

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = save_model_name, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
messages = [
    {"role": "user", "content": "Give me a level with many coins, many powerups"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# _ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
#                    use_cache = True, temperature = 1.5, min_p = 0.1)

outputs = model.generate(input_ids = inputs, max_new_tokens = 512, use_cache = True,
                         temperature = 1.2, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGive me a level with many coins, many powerups<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n--------------------------------------------------\n--------------------------------------------------\n--------------------------------------------------\n--------------------------------------------------\n--------------------------------------------------\n------------------SSSSS---------------------------\n-----------------------ooo--?-----ooo----ooo-----\n-----------------R---g-g-g---------------------##\n--<>--------------S!SS!SS!S--------------------###\n--[]-------------------------------------------####\n--[]-------------------------------------------#####\n--[]------XXXXX---XXXXX-XX-----XX-----XXXXXXXXXXXX\n--[]------XXXXX---XXXXX-XX-----XX-----XXXXXXXXXXXX<|eot_id|>']

#### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")


Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!
